<a href="https://colab.research.google.com/github/CS418/group-project-blackbox/blob/main/Spotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Project Introduction**
Predicting the popularity of songs based purely on song metrics such as key, danceability, and acousticness. Currently, being able to predict that something might be popular beforehand is an important research subject for every industry. It also has recently become a very important subject for the growing and competitive music industry as well. Since wide use of digital music platforms (Spotify, Billboard, Lastfm), data can be easily reached and the listening behaviors of the listeners can be easily observed. This provides convenience in forecasting techniques and it is also frequently used in recommendation systems.